### DL-LA with CNN Test

In [2]:
#import python libraries
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Input, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.models import load_model
from keras.layers import Dropout
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

from tensorflow.python.client import device_lib
import os

# Only use GPU1 - Quadro P6000
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
#import python libraries
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Input, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, AveragePooling1D,BatchNormalization
from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras.models import load_model
from keras.layers import Dropout
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
import operator as op
from functools import reduce
from tqdm import tnrange,tqdm_notebook
import operator as op
from functools import reduce
import os
import scipy.io as sio

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
d_len = 5083400
d_file_len = 5083400
num_files = 50834

In [5]:
def concatDataTraces(path):
    mats = []
    index = 0 
    fileNames = np.arange(100, (d_len + 100), 100).tolist()
    for r in range(0,num_files):
        mats.append(sio.loadmat(path+"traces_"+ str(fileNames[index]) +".mat", verify_compressed_data_integrity=False))
        index = index + 1
    mats = np.array(mats)
    myArray = []
    
    for i in range(0,num_files):
        for j in range(0,10):
            myArray.append(mats[i]['data'][j])
    return(myArray)

In [6]:
def concatDataLabel(path):
    mats = []
    index = 0 
    fileNames = np.arange(100, (d_len + 100), 100).tolist()
    for r in range(0,num_files):
        mats.append(sio.loadmat(path+"labels_"+ str(fileNames[index]) +".mat", verify_compressed_data_integrity=False))
        index = index + 1
    mats = np.array(mats) 
    myArray = []
    for i in range(0,num_files):
        for j in range(0,10):
            myArray.append(mats[i]['data'][0][j])
    return(myArray)

In [7]:
df_data = concatDataTraces('../real_traces/SBOX_traces2019-12-24_10_54_27/spot_4_12/sliced/')

In [8]:
#df_data = df_data[0:10000]

In [9]:
len(df_data[0])

5000

In [10]:
len(df_data)

508340

In [11]:
len(df_data)

508340

In [12]:
for i in range(0,len(df_data)):
    df_data[i] = df_data[i]

In [13]:
len(df_data)

508340

In [14]:
counter = 0
df_new_label = []
for i in range(0,len(df_data)):
    if counter == 0:
        df_new_label.append(0)
        counter = 1
    else:
        df_new_label.append(1)
        counter = 0

In [15]:
len(df_new_label)

508340

In [16]:
df_label = df_new_label

In [17]:
df_data = np.array(df_data)

In [18]:
df_data = np.reshape(df_data,(df_data.shape[0], df_data.shape[1], 1))

In [19]:
x_trainA, x_validA, y_trainA, y_validA = train_test_split(df_data,df_label,test_size=0.2,shuffle=True,stratify=df_label)

In [20]:
x_trainAC, x_validAC, y_trainAC, y_validAC = train_test_split(x_trainA,y_trainA,test_size=0.2,shuffle=True,stratify=y_trainA)

In [21]:
len(x_trainA)

406672

In [22]:
len(x_trainA) + len(x_validA)

508340

### CNN

In [23]:
### CNN Best model
from keras.layers.normalization import BatchNormalization

def createCNNModel(classes=2):
    # From VGG16 design
    input_shape = (5000,1)
    img_input = Input(shape=input_shape)
    # Block 1
    x = Conv1D(filters=120, kernel_size=8, activation='relu', name='block1_conv1')(img_input)
    x = BatchNormalization()(x)
    #x = MaxPooling1D(2)(x)

    x = Conv1D(filters=60, kernel_size=4, activation='relu', name='block2_conv1')(x)
    x = BatchNormalization()(x)
    #x = MaxPooling1D(2)(x)
    
    x = Conv1D(filters=20, kernel_size=4, activation='relu', name='block3_conv1')(x)
    x = BatchNormalization()(x)
    #x = MaxPooling1D(2)(x)
    
    x = Flatten(name='flatten')(x)

    x = Dense(classes, activation='softmax', name='predictions')(x)

    inputs = img_input
    
    # Create model.
    model = Model(inputs, x, name='cnn_best')
    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1/100000000, decay=0.0, amsgrad=False)
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    return model

In [24]:
tmpModel = createCNNModel()

In [ ]:
trainedModel = tmpModel.fit(x=[x_trainA],
                            y=to_categorical(y_trainA,num_classes=2),
                            validation_data=([x_validAC],to_categorical(y_validAC,num_classes=2)),
                            batch_size=1000,
                            verbose=1,
                            epochs=30
                            )


Train on 406672 samples, validate on 81335 samples
Epoch 1/30
  4000/406672 [..............................] - ETA: 1:35:58 - loss: 0.4782 - accuracy: 0.5163

In [ ]:
trainedModel

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(trainedModel.history['accuracy'])
plt.plot(trainedModel.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(trainedModel.history['loss'])
plt.plot(trainedModel.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(trainedModel.history['accuracy'])
plt.plot(trainedModel.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(trainedModel.history['loss'])
plt.plot(trainedModel.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(trainedModel.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(trainedModel.history['loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
def M(validation_data):
    return(len(validation_data))

In [ ]:
def sM(v,M):
    return(int(v*M))

In [ ]:
def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom

In [ ]:
from fractions import Fraction

def P(sM,M):
    term1 = 0.5**M
    allData = []
    for k in tnrange(sM,M,1):
        tmpCalc = ncr(M,k)
        tmp = term1 * tmpCalc
        allData.append(tmp)
    return(allData)

In [ ]:
myAcc = accuracy_score(y_validA,pedicatiosArray)
myM = M(x_validA)
mySM = sM(myAcc,myM)

In [ ]:
myP = P(mySM,myM)

In [ ]:
myP

In [ ]:
for i in myP:
    pTH = 10**-5
    if i < pTH:
        print("true")

In [ ]:
from scipy.stats import binom

In [ ]:
binom(myM,0.5)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(- np.log10(myP),'-b')
plt.axhline(y=5.0, xmin=0, xmax=1, color='r', linestyle='-', lw=1)
plt.show()